<img src='../../media/common/LogoWekeo_Copernicus_RGB_0.png' align='left' height='96px'></img>

<hr>

## Prepare input data for training a supervised learning model

*Authors: Adrian Di Paolo, Chung-Xiang Hong, Jonas Viehweger* 

This notebook shows the steps towards preparing data for training a supervised machine learning model. We will train the model based on the [pan-European High Resolution Vegetation Phenology and Productiviy (HR-VPP)](https://collections.sentinel-hub.com/vegetation-phenology-and-productivity-parameters-season-1/) data, which is derived from ESA’s Sentinel-2 as part of the Copernicus Land Monitoring Service (CLMS). The 13 parameters that describe specific stages of the seasonal vegetation growth cycle will be used as input features to fit a model.

As for the ground truth, we will use the [EuroCrops](https://github.com/maja601/EuroCrops#vectordata_zenodo) data, which is a dataset combining all publicly available self-declared crop reporting datasets from countries of the European Union.

In this example notebook, the expected outcome is a binary classifier that identifies grassland and non-grassland areas in the Netherlands. We will start by preparing a small dataset to train models with different algorithms, so we can have an overview on the performance of the models that is essential in the following model selection process.

Preparing the data at small scale for training a model can be simple and straightforward with the following three steps:
1. [Define training and validation area](#1-define-training-and-validation-area)
2. [Obtain features and labels](#2-obtain-features-and-labels)
3. [Reshape data for model training](#3-reshape-data-for-model-training)



In [ ]:
# Necessary imports

library(fs)
library(dplyr)

library(sf)
library(terra)
library(stars)
library(purrr)

library(leaflet)
library(ggplot2)

library(hdar)

## 1. Define training and validation area

To train a machine learning model, we need to define a training dataset and a validation dataset. The training data set is a set of example data which is used to fit the parameters (e.g., weights) during the learning process. Then, the trained model needs to be evaluated using examples from the held-out dataset, which is the validation dataset that is independent of the training dataset and is not used in the training process.

In our case, we will randomly select a training area and a validation area in the Netherlands to obtain examples for the training and the evaluation process. First let's take a look at these two areas.

In [ ]:
# define bounding box
bounds_poly <- st_read("../../data/raw/ml-grassland-classification/bounding_boxes.geojson")
validation_gdf <- bounds_poly[nrow(bounds_poly), ]
train_gdf <- bounds_poly[-nrow(bounds_poly), ]

In [ ]:
# Transform the data to WGS84 (EPSG:4326)
total_bbox <- st_transform(bounds_poly, crs = 4326) %>% st_bbox()
validation_bbox <- st_transform(validation_gdf, crs = 4326) %>% st_bbox()
train_bbox <- st_transform(train_gdf, crs = 4326) %>% st_bbox()

print(train_bbox)

In [ ]:
# Create the map
map <- leaflet() %>%
  addTiles() %>%
  setView(lng = 7.014084, lat = 53.104538500000004, zoom = 8)

# Add validation geometries in red
map <- map %>%
  addPolygons(data = st_transform(validation_gdf, crs = 4326), color = "red", weight = 2)

# Add training geometries in blue
map <- map %>%
  addPolygons(data = st_transform(train_gdf, crs = 4326), color = "blue", weight = 2)

map


### Explore the ground truth dataset

Since we are going to train a binary classification model to identify if an area is grassland or not in the Netherlands, it is important to investigate the distribution of crop types in the country.  

Here we display the distribution of the top 10 categories reported in the Netherlands. We can see that grassland is the most dominant category in the data, which is a critical point we should pay attention to when sampling data for training at a national scale.

**Note** that [EuroCrops Netherlands](https://zenodo.org/record/7476474/files/NL_2020.zip?download=1) data needs to be downloaded and saved in the same folder where this notebook is located.

This is also quite a big dataset, so loading it entirely may take a while.

In [ ]:
# load full dataset to GeoDataFrame
EC_NETHERLANDS_PATH <- "../../data/processing/ml-grassland-classification/NL/NL_2020_EC21.shp"
netherlands_gt <- st_read(EC_NETHERLANDS_PATH)

In [ ]:
knitr::kable(head(netherlands_gt))

In [ ]:
categories <- netherlands_gt %>%
              st_drop_geometry %>%
              count(EC_hcat_n, sort = TRUE)

# create a subset for the top 10 catagories
categories_subset <- categories %>% slice_head(n = 10)

In [ ]:
# Adjust the plot size
options(repr.plot.width = 24, repr.plot.height = 12)

ggplot(categories_subset, aes(x = "", y = n, fill = EC_hcat_n)) +
  geom_bar(width = 1, stat = "identity") +
  coord_polar(theta = "y") +
  theme_void() +
  theme(legend.position = "right", 
      legend.text = element_text(size = 20),
      legend.title = element_text(size = 24),
      legend.margin = margin(0, 24, 0, 0),
  ) +
  geom_text(aes(label = scales::percent(n / sum(n), accuracy = 0.1)),
    position = position_stack(vjust = 0.5), size = 5,
  ) +
  labs(fill = "Categories")


Next, let's have a closer look at the data in the training area and the validation area. We can see the distribution of crop types is much more balanced in both the training area and validation area.

In [ ]:
train_gt <- st_read(EC_NETHERLANDS_PATH, wkt_filter = st_as_text(st_as_sfc(train_bbox)))

In [ ]:
# Adjust the plot size
options(repr.plot.width=14, repr.plot.height=12)

# plot all geometries in the training area
ggplot() +
  geom_sf(data = train_gt) +
  theme_minimal() +
  theme(axis.text = element_text(size = 14))

In [ ]:
# create pie chart
top_categories <- train_gt %>%
  st_drop_geometry() %>% # Remove geometry for counting
  count(EC_hcat_n, sort = TRUE) %>%
  slice_head(n = 10)

options(repr.plot.width = 20, repr.plot.height = 14)

ggplot(top_categories, aes(x = "", y = n, fill = EC_hcat_n)) +
  geom_bar(width = 1, stat = "identity") +
  coord_polar("y") +
  theme_void() +
  theme(legend.position = "right", 
        legend.text = element_text(size = 20),
        legend.title = element_text(size = 24),
        legend.margin = margin(0, 24, 0, 0)
  ) +
  geom_text(aes(label = scales::percent(n / sum(n), accuracy = 0.1)),
    position = position_stack(vjust = 0.5), size = 5
  ) +
  labs(fill = "Top Categories")


In [ ]:
validation_gt <- st_read(EC_NETHERLANDS_PATH, wkt_filter = st_as_text(st_as_sfc(validation_bbox)))

In [ ]:
# Adjust the plot size
options(repr.plot.width=14, repr.plot.height=12)

# plot all geometries in the training area
ggplot() +
  geom_sf(data = validation_gt) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 20),
    axis.title = element_text(size = 16),
    axis.text = element_text(size = 14)
  )

In [ ]:
# Count unique elements in EC_hcat_n and get the top 10
top_categories_validation <- validation_gt %>%
  st_drop_geometry() %>%  # Remove geometry for counting
  count(EC_hcat_n, sort = TRUE) %>%
  slice_head(n = 10)

options(repr.plot.width = 20, repr.plot.height = 14)

# Create the pie chart
ggplot(top_categories_validation, aes(x = "", y = n, fill = EC_hcat_n)) +
  geom_bar(width = 1, stat = "identity") +
  coord_polar(theta = "y") +
  theme_void() +
  theme(legend.position = "right", 
        legend.text = element_text(size = 16),
        legend.title = element_text(size = 20),
        legend.margin = margin(0, 24, 0, 0)
  ) +
  geom_text(aes(label = scales::percent(n / sum(n), accuracy = 0.1)),
    position = position_stack(vjust = 0.5), size = 5
  ) +
  labs(fill = "Categories")



### Bias

Training and validating a model with samples in small areas will introduce a bias. The bias comes from two aspects:
* The imbalanced samples of grassland and non-grassland
* The spatial correlation between samples

This will be addressed later when preparing data at a large scale to train a model general enough for the whole country.

## 2. Obtain HR-VPP data and labels

In total we will show two possibilities to get HR-VPP data. The first is by using the HDA API which is available on WEkEO. This API has a few limitations which necessitate a bit more post-processing of the data. In a later notebook we will show how to access the data using the Sentinelhub API which is more capable in its processing.

### Downloading Data

First the data is downloaded from WEkEO using the HDA R package. A getting started tutorial for the API is available [here](https://help.wekeo.eu/en/articles/6751608-what-is-the-hda-api-python-client-and-how-to-use-it). 

For the download we specify a helper function which subsets the data query to only a single tile. This is done to reduce the amount of data that will be downloaded, since a lot of tiles are overlapping the study area. The HDA API will always download entire tiles even if you are only interested in small parts of the tile.

In [ ]:
hda_tile_download <- function(client, query, tile, folder) {
  matches <- client$search(query)

  matches_subset <- Filter(function(match) grepl(tile, match$properties$location), matches$results)
  matches$results <- matches_subset

  matches$download(folder, prompt = FALSE)
}

In the query we specify which dataset we want to download, for which timeframe and for which area.

The HDA API downloads different HR-VPP parameters in separate TIFF files. So the next step is to read in the data for our training and validation areas and stack them together. To ensure you download all the necessary parameters, remember to adjust the “productType” with the specific parameter IDs: `'AMPL', 'EOSD', 'EOSV', 'LENGTH', 'LSLOPE', 'MAXD', 'MAXV', 'MINV', 'RSLOPE', 'SOSD', 'SOSV', 'SPROD', 'TPROD'`. This will ensure the correct set of HR-VPP data is included for your analysis.

In [ ]:
c <- Client$new()

product_types <- list('AMPL', 'EOSD', 'EOSV', 'LENGTH', 'LSLOPE', 'MAXD', 'MAXV', 'MINV', 'QFLAG', 'RSLOPE', 'SOSD', 'SOSV', 'SPROD', 'TPROD')

for (product_type in product_types) {

  query_obj <- list(
    dataset_id = "EO:EEA:DAT:CLMS_HRVPP_VPP",
    productType = product_type,
    productVersion = "V101",
    productGroupId = "s1",
    start = "2020-01-01T00:00:00.000Z",
    end = "2020-01-01T00:00:00.000Z",
    bbox = as.numeric(total_bbox)
  )

  query <- jsonlite::toJSON(query_obj, auto_unbox = TRUE)
  hda_tile_download(c, query, tile = "T32ULD", folder = "../../data/download/ml-grassland-classification/hda")
}

In [ ]:
dir_path <- "../../data/download/ml-grassland-classification/hda/"
file_list <- list.files(path = dir_path, pattern = "\\.tif$", full.names = TRUE)
file_list <- sort(file_list)

print(file_list)

In [ ]:
read_raw_raster <- function(file_path) {

  float_values <- rast(file_path)

  # Get the scale factor and offset using scoff
  scale_offset <- scoff(float_values)

  # Initialize scale and offset
  scale_factor <- scale_offset[[1]]
  offset <- scale_offset[[2]]

  # Adjust the values based on scale and offset if present
  raw_values <- (float_values - offset) / scale_factor

  round(raw_values)
}

get_affine_transform <- function(src) {
  # Extract extent and resolution
  ext <- ext(src)
  res <- res(src)

  # Calculate affine transformation components
  xmin <- ext[1]
  ymax <- ext[4]
  xres <- res[1]
  yres <- res[2]

  # Create the affine transformation matrix
  matrix(c(xres, 0, xmin, 0, -yres, ymax, 0, 0, 1), nrow = 3, byrow = TRUE)
}

compute_window <- function(transform, bbox) {
  # Extract affine transform components
  xres <- transform[1, 1]
  yres <- abs(transform[2, 2]) # Ensure positive resolution for calculation
  xmin <- transform[1, 3]
  ymax <- transform[2, 3]

  # Calculate pixel coordinates for the bounding box
  col_min <- floor((bbox["xmin"] - xmin) / xres)
  col_max <- floor((bbox["xmax"] - xmin) / xres)
  row_min <- floor((ymax - bbox["ymax"]) / yres)
  row_max <- floor((ymax - bbox["ymin"]) / yres)

  # Create window as list of coordinates
  window <- list(col_min = col_min, col_max = col_max, row_min = row_min, row_max = row_max)
}

crop_raster_with_bbox <- function(src, bbox) {
  # Extract extent and resolution
  ext <- ext(src)
  res <- res(src)

  # Calculate affine transformation components
  xmin <- ext[1]
  ymax <- ext[4]
  xres <- res[1]
  yres <- res[2]

  # Calculate pixel coordinates for the bounding box
  col_min <- floor((bbox["xmin"] - xmin) / xres)
  col_max <- floor((bbox["xmax"] - xmin) / xres)
  row_min <- floor((ymax - bbox["ymax"]) / yres)
  row_max <- floor((ymax - bbox["ymin"]) / yres)

  # Calculate spatial coordinates for the window extent
  win_extent <- ext(c(xmin + col_min * xres, xmin + col_max * xres, ymax - row_max * yres, ymax - row_min * yres - 1))

  # Crop the raster using the window extent
  crop(src, win_extent, snap = "out")
}

compute_new_transform <- function(bounds, window) {
  width <- window$col_max - window$col_min
  height <- window$row_max - window$row_min

  # Extract bounding box components
  xmin <- bounds["xmin"]
  ymin <- bounds["ymin"]
  xmax <- bounds["xmax"]
  ymax <- bounds["ymax"]

  # Calculate the resolution
  xres <- ceiling((xmax - xmin) / width)
  yres <- ceiling((ymax - ymin) / height)

  # Create the homogeneous affine transformation matrix (3x3)
  matrix(c(xres, 0, xmin, 0, -yres, ymax, 0, 0, 1), nrow = 3, byrow = TRUE)
}

compute_new_bounds <- function(transform, window) {
  # Extract affine transform components
  xres <- transform[1, 1]
  yres <- abs(transform[2, 2])
  xmin <- transform[1, 3]
  ymax <- transform[2, 3]

  # Extract window parameters
  col_min <- window$col_min
  col_max <- window$col_max
  row_min <- window$row_min
  row_max <- window$row_max

  # Convert window pixel coordinates to geographic coordinates
  x_min <- xmin + col_min * xres
  x_max <- xmin + col_max * xres
  y_max <- ymax - row_min * yres
  y_min <- ymax - row_max * yres

  # Create the new bounding box
  c(x_min, y_min, x_max, y_max)
}

stack_hrvpp <- function(file_list, gdf) {
  arrays <- list()
  new_transforms <- list()
  bounds <- st_bbox(gdf)

  for (file in file_list) {

    src <- read_raw_raster(file)
    transform <- get_affine_transform(src)
    window <- compute_window(transform, bounds)

    cropped_raster <- crop_raster_with_bbox(src, bounds)

    arrays[[length(arrays) + 1]] <- as.matrix(cropped_raster, wide = TRUE)

    new_bounds <- compute_new_bounds(transform, window)
    new_transform <- compute_new_transform(new_bounds, window)
    new_transforms[[length(new_transforms) + 1]] <- new_transform

    # Remove temporary files created by terra
    terra::tmpFiles(remove = TRUE)
  }

  stacked_array <- abind::abind(arrays, along = 3)

  return(list(stacked_array, new_transforms[[1]]))
}


In [ ]:
train_result <- stack_hrvpp(file_list, train_gdf)
train_data <- train_result[[1]]
train_transform <- train_result[[2]]

validation_result <- stack_hrvpp(file_list, validation_gdf)
validation_data <- validation_result[[1]]
validation_transform <- validation_result[[2]]

We now have two arrays, one for the training area and one for the validation area, which is smaller than the one for the training area. But both arrays now have all 14 bands as their third dimension.

### Get labels

We have the HR-VPP data of both training and validation areas now. Next, we want to get the labels for these two areas as the ground truth to perform a supervised training.

To do this, we have to convert the vector data to raster data. This is done with the `get_labels` function which sets up the data for the `rasterio` function `rasterize()`.

In [ ]:
get_crs <- function(tiff_path) {
  src <- rast(tiff_path)
  crs <- crs(src, describe=TRUE, proj=TRUE)
  return(paste0(crs$authority, ":", crs$code))
}

crs <- get_crs(file_list[[1]])

In [ ]:
get_labels <- function(ec_gt, geotransform, out_shape, raster_crs) {

  # Convert the input CRS to the raster CRS
  ec_gt <- st_transform(ec_gt, crs = raster_crs)
  
  # Convert the EC_hcat_c column to numeric
  ec_gt$label <- as.numeric(ec_gt$EC_hcat_c)

  ec_gt <- ec_gt %>% select(geometry, label)
  
  # Extract the geotransform parameters
  x_min <- geotransform[1, 3]
  y_max <- geotransform[2, 3]
  x_res <- geotransform[1, 1]
  y_res <- -geotransform[2, 2]
  
  # Define the dimensions
  nrows <- out_shape[1]
  ncols <- out_shape[2]

  # Calculate the extent of the raster
  x_max <- x_min + ncols * x_res
  y_min <- y_max - nrows * y_res
  
  # Define the extent and resolution
  extent <- st_bbox(c(xmin = x_min, ymin = y_min, xmax = x_max, ymax = y_max), crs = raster_crs)

  #raster_array <- array(-1, dim = c(ncols, nrows))

  raster_template <- st_as_stars(extent, nx = ncols, ny = nrows)
  raster_template[[1]] <- array(-1, dim = c(ncols, nrows))
  
  # Rasterize the vector data
  rasterized <- st_rasterize(ec_gt, template = raster_template, values = "label", dx = x_res, dy = y_res)
}


In [ ]:
train_labels <- get_labels(train_gt, train_transform, dim(train_data)[1:2], crs)
train_labels <- as.matrix(train_labels[[1]])

validation_labels <- get_labels(validation_gt, validation_transform, dim(validation_data)[1:2], crs)
validation_labels <- as.matrix(validation_labels[[1]])

In [ ]:
plot_raster_with_legend <- function(rast, title) {
  nrows <- nrow(rast)
  ncols <- ncol(rast)

  # Create a data frame with x, y, and value columns
  train_labels_df <- data.frame(
    y = rep(1:ncols, each = nrows),
    x = rep(1:nrows, ncols),
    value = as.vector(rast)
  )

  # Categorize the values
  train_labels_df$category <- ifelse(train_labels_df$value > 0, "Grassland", "Non-Grassland")

  # Plot the matrix using ggplot2
  ggplot(train_labels_df, aes(x = x, y = y, fill = category)) +
    geom_tile() +
    scale_fill_manual(values = c("Grassland" = "#308c30", "Non-Grassland" = "#adaaaa")) + 
    coord_equal() +
    scale_y_reverse() +
    theme_minimal() +
    theme(
      plot.title = element_text(size = 20),
      axis.title = element_text(size = 16),
      axis.text = element_text(size = 14),
      legend.text = element_text(size = 20),
      legend.title = element_text(size = 24),
      legend.margin = margin(0, 24, 0, 0)
    ) +
    labs(title = title, x = "Column Index", y = "Row Index", fill = "Value")
}

plot_raster_with_legend(train_labels, "Raster Plot with Legend for Training Data")
plot_raster_with_legend(validation_labels, "Raster Plot with Legend for Validation Data")

## 3. Reshape data for model training

At this point we have all the data we need to train a supervised machine learning model. The last step is to reshape the data so that it can be fed into the model. In general, we simply flatten the arrays to go from 3 dimensional data to 2 dimensional data, where the first dimension are the pixels and the second dimension the values of the bands.

In [ ]:
get_model_input <- function(features, labels) {

  # Reshape the features array
  x <- array(aperm(features, c(2, 1, 3)), dim = c(dim(features)[1] * dim(features)[2], dim(features)[3]))
  
  # Flatten the labels array
  y <- as.vector(labels)
  
  # Filter out rows where the corresponding label is -1
  x_clean <- x[y != -1, , drop = FALSE]
  y_clean <- y[y != -1]
  
  list(x_clean = x_clean, y_clean = y_clean)
}


In [ ]:
train_model_input <- get_model_input(train_data, train_labels)
x_data <- train_model_input$x_clean
y_data <- train_model_input$y_clean

In [ ]:
validation_model_input <- get_model_input(validation_data, validation_labels)
x_validation <- validation_model_input$x_clean
y_validation <- validation_model_input$y_clean

In [ ]:
# List of datasets
datasets <- list(
  x_validation = x_validation,
  y_validation = y_validation,
  x_data = x_data,
  y_data = y_data
)

# Create directory if it doesn't exist
dir_path <- "../../data/processing/ml-grassland-classification/dataset"
dir_create(dir_path)

# Save each dataset to .rds files
for (name in names(datasets)) {
  dataset <- datasets[[name]]
  saveRDS(dataset, file.path(dir_path, paste0(name, ".rds")))
}

## Conclusion



When preparing the input data for a supervised learning task, there are a few important points which can be the takeaways from this notebook:  
1. Ground truth data is critical for a supervised learning task. It determines how well a model can learn from your data and the reliability of the validation result.
2. It is important to have two separate dataset. One for training and the other for validation. Always make sure that the validation dataset is held-out in the training process.
3. Training with a limited amount of samples can lead to bias. The bias can be alleviated if there are sufficient amounts of samples.  